In [1]:
import pandas as pd
import arcgis
from arcgis.gis import GIS
from arcgis.features import GeoAccessor,GeoSeriesAccessor
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from arcgis.mapping import WebMap
from IPython.display import display

In [2]:
df = pd.read_csv('data/kc_house_data.csv')

gis = GIS("http://www.arcgis.com/", "shadowsword_0","Acidblade1")

In [15]:
def gis_content():
    item = gis.content.get('ff9e4774ee8641f48cccac97dca753db#data')
    flayer = item.layers[0]
    food = flayer.query(where = "ESITE = '0000000'").sdf.copy()
    item = gis.content.get('3f263039314d44cc93384fe1f4229796#data')
    flayer = item.layers[0]
    public = flayer.query(where = "ZIPCODE > 0").sdf.copy()
    item = gis.content.get('0dfe37d2a68545a699b999804354dacf')
    flayer = item.layers[0]
    private = flayer.query(where = "STATE = 'WA'").sdf.copy()
    
    #Editing data for later use
    public_df = public.drop(['FID','PIN','SCH_CLASS','CODE','FEATURE_ID','ESITE','MAJOR','MINOR','SHAPE'],1)
    private_df = private.drop(['FID','NCESID','ZIP4','VAL_METHOD','VAL_DATE','SHELTER_ID','COUNTRY','SOURCE','SOURCEDATE','NAICS_CODE','FT_TEACHER','START_GRAD','END_GRADE','COUNTYFIPS','SHAPE','TYPE','STATUS','VAL_METHOD','VAL_DATE'],1)
    food_df = food.drop(['RECORD_ID','FACILITY_NAME','CHAIN_NAME','CHAIN_ESTABLISHMENT','SITE_ADDRESS','ABB_NAME','ESITE','FEATURE_ID'],1)
    private_df = private_df.rename(columns ={'ZIP':'zipcode','LONGITUDE':'long',':LATITUDE':'lat'})
    public_df = public_df.rename(columns = {'LONG_CEN': 'LONGITUDE', 'LAT_CEN': 'LATITUDE','ZIPCODE':'zipcode'})
    private_df['Private'] = 'yes'
    public_df['Private'] = 'No'
    public_df['zipcode'] = public_df['zipcode'].astype(str).astype(int)
    private_df['zipcode'] = private_df['zipcode'].astype(str).astype(int)
    public_df['Private'].astype(str)
    return (food_df,public_df,private_df) 


In [16]:
food,public,private = gis_content()

In [22]:
def mapping(df):
    """Taking in a dataframe to assign the budget through all fields. Takes schools into account as well. If not looking for schools
        just enter through"""
#     mapp = gis.content.get('579a5ea9a24d4b85b237ef8e9cb578b4')
    budget = int(input('What is your budget: '))
#     if budget is None:
#         budget_df = df
    schools = input('Are you interested in schools? ')
    budget_df = df.loc[df.price<=budget].copy()
    budget_df.is_copy = None
    pdx_map = gis.map('Seattle WA')
    if (schools == 'Yes') or (schools == 'yes') and (budget is not None):
        interest = input('Private school,Public school or both?')
        if (interest =='Private') or (interest =='private'):
#             private_df = budget_df.merge(private, how = 'outer')
#             private_df.fillna(0)
#             private_df.set_index('zipcode')
            school_sdf = pd.DataFrame.spatial.from_xy(private,'long','lat')
            school_sdf.spatial.plot(map_widget = pdx_map,renderer_type = "u",symbol_type = 'simple',symbol_style = 'o',title = 'Schools', 
                    col='zipcode',
                    cmap='spring',  # matplotlib color map
                    alpha=0.7,
                    size = .5, 
                    outline_color=[0,0,0,0])
        elif(interest =='Public') or (interest == 'public'):
            school_sdf = pd.DataFrame.spatial.from_xy(public,'LONGITUDE','LATITUDE')
            school_sdf.spatial.plot(map_widget = pdx_map,renderer_type = "u",symbol_type = 'simple',symbol_style = 'o',title = 'Schools', 
                    col='zipcode',
                    cmap='spring',  # matplotlib color map
                    alpha=0.7,
                    size = .5, 
                    outline_color=[0,0,0,0])
        else:
            school_sdf = pd.DataFrame.spatial.from_xy(all_schools,'LONGITUDE','LATITUDE')
            school_sdf.spatial.plot(map_widget = pdx_map,renderer_type = "u",symbol_type = 'simple',symbol_style = 'o',title = 'Schools', 
                      col='zipcode',
                      cmap='spring',  # matplotlib color map
                    alpha=0.7,
                    size = .5, 
                    outline_color=[0,0,0,0])
        
       
    data_sdf = pd.DataFrame.spatial.from_xy(budget_df, 'long','lat')
    
       
    data_sdf.spatial.plot(map_widget=pdx_map, renderer_type = "c",marker_size = 5
                    ,symbol_type = 'simple',symbol_style='d',
                    title='Pricing of houses',
                    col='price',
                    cmap='spring',  # matplotlib color map
                    alpha=0.7,
                    size = .5,
                    outline_color=[0,0,0,0]
                     )
#     food_sdf = pd.DataFrame.spatial(food)
#     food_sdf.plot(map_widget = pdx_map,renderer_type = "c",symbol_type = 'simple', symbol_style = 'x',
#                   title='Pricing of houses',
#                     col='price',  
#                   cmap='spring',  # matplotlib color map
#                     alpha=0.7,
#                     size = .5,
#                     outline_color=[0,0,0,0])
    pdx_map.legend = True
    display(pdx_map)
#     display(mapp)
# took legend out since the unique renderer_type makes it hard to assign general information through it. 
# Will work on getting that working so the distance from houses shows schools locally by zipcode.
# sns.barplot(x = 'price',y = 'sqft_living',data = budget_df)

In [24]:
mapping(df)

What is your budget: 150000
Are you interested in schools? yes
Private school,Public school or both?public


MapView(layout=Layout(height='400px', width='100%'), legend=True)

In [8]:
df.head()




,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,...,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,6,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


In [ ]:
# pull data from https://blue.kingcounty.com/Assessor/eRealProperty/Detail.aspx?ParcelNbr=
# adding the id number from each of the dataframes into the last bit of the code adding that to a new dataframe to work with the code